In [1]:
import pandas as pd
import sqlite3

In [2]:
# Create Database Connection
conn = sqlite3.connect('../HomelessData.db')
c = conn.cursor()

In [3]:
# Create HIC dataframe and database
for sheet in ["2018","2019"] : 
    hic_df = pd.read_excel("../resources/2007-2020-HIC-Counts-by-CoC.xlsx",sheet,skiprows=[0],skipfooter=1)
    
    # Adjust Column
    hic_df.rename(columns = {'CoC Number' : 'Coc','Total Year-Round Beds (ES, TH, SH)' : 'Total','Total Year-Round Beds (ES)' : 'Emergency','Total Year-Round Beds (TH)' : 'Transitional','Total Year-Round Beds (SH)' : 'SafeHaven','Total Year-Round Beds (RRH)' : 'RapidRehousing','Total Year-Round Beds (PSH)' : 'PermanentSupportive','Total Year-Round Beds (OPH)' : 'PermanentOther'},inplace=True)
    hic_df = hic_df[["Coc", "Total", "Emergency", "Transitional", "SafeHaven", "RapidRehousing", "PermanentSupportive", "PermanentOther"]]
    hic_df["Year"] = sheet
    
    # Insert Into DB
    hic_df.to_sql("shelterAvalibility",con=conn,if_exists = 'append',index=False)

In [4]:
# Create PIT and CityData dataframe and database
for sheet in ["2018","2019"] : 
    pit_df = pd.read_excel("../resources/2007-2020-PIT-Estimates-by-CoC.xlsx",sheet,skipfooter=3)
    
    # Rename pit_df columns
    pit_df.rename(columns = {'CoC Number' : 'Coc',f'Overall Homeless, {sheet}' : "Total",f'Unsheltered Homeless, {sheet}' : "Unsheltered"},inplace=True)
    
    #Create CityData Database
    pit_df[["Coc","CoC Name"]].to_sql("CityData",con=conn,if_exists = "replace")
    
    # Create New Columns
    pit_df["Sheltered"] = pit_df["Total"]-pit_df["Unsheltered"]
    pit_df = pit_df[['Coc','Total','Unsheltered','Sheltered']]
    pit_df["Year"] = sheet
    
    # Create HomelessData table
    pit_df.to_sql("HomelessCounts",con=conn,if_exists='append',index=False)

C:\Users\mupchurch\.conda\envs\PythonData\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [5]:
# Create The Funding Database and Dataframe
for sheet in ["2018","2019"] :
    awards_df = pd.read_excel("../resources/2019-Awards-by-CoC-Component-Project.xlsx",sheet)
    awards_df.rename(columns = {"CoC Number" : "CoC"},inplace=True)
    awards_df = awards_df[["CoC","Component","Awarded Amount"]]
    awards_df["Year"] = sheet
    awards_df.to_sql("FundingData",con=conn,if_exists='append')

In [6]:
hic_df.head()


,Coc,Total,Emergency,Transitional,SafeHaven,RapidRehousing,PermanentSupportive,PermanentOther,Year
0,AK-500,1033,799,234,0,171,601,71,2019
1,AK-501,845,575,270,0,67,386,0,2019
2,AL-500,827,516,277,34,106,1740,0,2019
3,AL-501,394,261,133,0,31,240,0,2019
4,AL-502,189,106,83,0,0,42,8,2019


In [7]:
output_file_path = "hic_summary.csv"
hic_df.to_csv(output_file_path, index=False)

In [8]:
CountyData_df.head()

NameError: name 'CountyData_df' is not defined